In [3]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/single-kmer-match

kmer_length=31

# Construct k-mer probe

In [5]:
grep 'ugd' kmer-probe/nucleotide_fasta_protein_homolog_model.fasta -A 1 | tee kmer-probe/ugd.fasta

>gb|U00096|-|2098446-2099613|ARO:3003577|ugd [Escherichia coli str. K-12 substr. MG1655] 
ATGAAAATCACCATTTCCGGTACTGGCTATGTAGGCTTGTCAAACGGGCTTCTAATCGCACAAAATCATGAGGTTGTGGCATTAGATATTTTACCGTCACGCGTTGCTATGCTGAATGATCGGATATCTCCTATTGTTGATAAGGAAATTCAGCAGTTTTTGCAATCAGATAAAATACACTTTAATGCCACATTAGATAAAAATGAAGCCTACCGGGATGCTGATTATGTCATCATCGCCACTCCAACCGACTATGATCCTAAAACTAATTATTTCAATACATCCAGTGTAGAATCAGTAATTAAAGACGTAGTTGAGATAAATCCTTATGCGGTTATGGTCATCAAATCAACGGTTCCCGTTGGTTTTACCGCAGCGATGCATAAGAAATATCGCACTGAAAATATTATATTCTCCCCGGAATTTCTCCGTGAGGGTAAAGCCCTTTACGATAATCTCCATCCTTCACGTATTGTCATCGGTGAGCGTTCAGAACGCGCAGAACGTTTCGCTGCTCTGTTACAGGAAGGCGCGATTAAGCAAAATATCCCGATGCTGTTTACCGACTCCACTGAAGCAGAAGCGATTAAACTTTTTGCAAACACCTACCTGGCGATGCGCGTGGCGTACTTTAACGAACTGGATAGCTATGCAGAAAGTTTAGGTCTGAATTCCCGTCAAATAATCGAAGGCGTTTGTCTCGACCCACGTATTGGCAACCATTACAACAATCCGTCGTTTGGTTATGGTGGTTATTGTCTGCCGAAAGATACCAAGCAGTTACTGGCGAACTACCAGTCTGTGCCGAATAACCTGATCTCGGCAATTGTCGATGCTAACCGCACGCGTAAAGATTTTATTGCCGATGCCATTTTGTCACGCAAGCCGCAAGTGGTGGGTATTTATCGTC

In [6]:
conda run --name jellyfish jellyfish count --size 10000000 --threads 8 --mer-len ${kmer_length} --output \
        kmer-probe/ugd.kmer.jf --canonical kmer-probe/ugd.fasta && \
    jellyfish dump --column kmer-probe/ugd.kmer.jf | cut -d ' ' -f 1 > kmer-probe/ugd.kmer

In [7]:
head kmer-probe/ugd.kmer
wc -l kmer-probe/ugd.kmer

AATGACGTCGGCTTGTTGTTTGAAGGTGGCG
AAAAATGAAGCCTACCGGGATGCTGATTATG
ACCAGTCTGTGCCGAATAACCTGATCTCGGC
CACTTCAACACCTTTCGCCTTGATACGTTTC
AGGTGTTTGCAAAAAGTTTAATCGCTTCTGC
AGCCCGTTTGACAAGCCTACATAGCCAGTAC
AATACGTGGGTCGAGACAAACGCCTTCGATT
GTGCGGTTAGCATCGACAATTGCCGAGATCA
GCCCTTTACGATAATCTCCATCCTTCACGTA
CCTACCGGGATGCTGATTATGTCATCATCGC
1137 kmer-probe/ugd.kmer


Pick a single kmer from this list.

In [12]:
(echo ">ugd-single-kmer"; head -n 1 kmer-probe/ugd.kmer) | tee kmer-probe/ugd.single.fasta

>ugd-single-kmer
AATGACGTCGGCTTGTTGTTTGAAGGTGGCG


# Pull out reads matching kmer

In [13]:
conda run --name kat /usr/bin/time -v kat filter seq -t 1 -o SRR1952908.kat -m ${kmer_length} \
    --seq ../input/SRR1952908_1.fastq --seq2 ../input/SRR1952908_2.fastq \
    kmer-probe/ugd.single.fasta

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in filter sequence mode
-----------------------------------

Input 49712 is a sequence file.  Counting kmers for input 49712 (kmer-probe/ugd.single.fasta) ... done.  Time taken: 0.1s

Filtering sequences ...
Processed 100000 pairs
Processed 200000 pairs
Processed 300000 pairs
Processed 400000 pairs
Processed 500000 pairs
Processed 600000 pairs
Processed 700000 pairs
Processed 800000 pairs
Processed 900000 pairs
Processed 1000000 pairs
Processed 1100000 pairs
Processed 1200000 pairs
Processed 1300000 pairs
Processed 1400000 pairs
Processed 1500000 pairs
Processed 1600000 pairs
Processed 1700000 pairs
Processed 1800000 pairs
Processed 1900000 pairs
Processed 2000000 pairs
Processed 2100000 pairs
Finished filtering.  Time taken: 107.9s

Found 0 / 2147046 to keep

KAT filter seq completed.
Total runtime: 108.0s

	Command being timed: "kat filter seq -t 1 -o SRR1952908.kat -m 31 --seq ../input/SRR1952908_1.fastq --seq2 ../input/SRR1952908_2.fa

In [14]:
ls -lrth

total 12K
drwxr-xr-x 2 apetkau grp_apetkau 4.0K Jan 17 10:10 kmer-probe
-rw-r--r-- 1 apetkau grp_apetkau    0 Jan 17 10:10 SRR1952908.kat.in.R2.fastq
-rw-r--r-- 1 apetkau grp_apetkau    0 Jan 17 10:10 SRR1952908.kat.in.R1.fastq
-rw-r--r-- 1 apetkau grp_apetkau 4.3K Jan 17 10:10 assmebly-kmer-analysis.ipynb
